In [6]:
import random
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter

import torchvision.utils as vutils

import gym
import gym.spaces

import numpy as np

In [7]:
log = gym.logger
log.set_level(gym.logger.INFO)

LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

# dimension input image will be rescaled
IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

CUDA = True

In [8]:
class InputWrapper(gym.ObservationWrapper):
    """
    Preprocessing of input numpy array:
    1. resize image into predefined size
    2. move color channel axis to a first place
    """
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args)
        assert isinstance(self.observation_space, gym.spaces.Box)
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(self.observation(old_space.low), self.observation(old_space.high),
                                                dtype=np.float32)

    def observation(self, observation):
        # resize image
        new_obs = cv2.resize(observation, (IMAGE_SIZE, IMAGE_SIZE))
        # transform (210, 160, 3) -> (3, 210, 160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)

In [9]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # this pipe converges image into the single number
        self.conv_pipe = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 2, out_channels=DISCR_FILTERS * 4,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 4, out_channels=DISCR_FILTERS * 8,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 8, out_channels=1,
                      kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        conv_out = self.conv_pipe(x)
        return conv_out.view(-1, 1).squeeze(dim=1)

In [10]:
class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # pipe deconvolves input vector into (3, 64, 64) image
        self.pipe = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS * 8,
                               kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(GENER_FILTERS * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 8, out_channels=GENER_FILTERS * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 4, out_channels=GENER_FILTERS * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.pipe(x)


In [11]:
def iterate_batches(envs, batch_size=BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)

    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalising input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32) * 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

In [20]:
device = torch.device("cuda" if CUDA else "cpu")
envs = [InputWrapper(gym.make(name)) for name in ('Breakout-v0', 'Assault-v0', 'Pong-v0')]
input_shape = envs[0].observation_space.shape

net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

objective = nn.BCELoss()
gen_optimizer = optim.Adam(params=net_gener.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
dis_optimizer = optim.Adam(params=net_discr.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
writer = SummaryWriter()

gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, dtype=torch.float32, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, dtype=torch.float32, device=device)

for batch_v in iterate_batches(envs):
    # generate extra fake samples, input is 4D: batch, filters, x, y
    gen_input_v = torch.FloatTensor(BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1).normal_(0, 1).to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)

    # train discriminator
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_true_v, true_labels_v) + objective(dis_output_fake_v, fake_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    dis_losses.append(dis_loss.item())

    # train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss_v = objective(dis_output_v, true_labels_v)
    gen_loss_v.backward()
    gen_optimizer.step()
    gen_losses.append(gen_loss_v.item())

    iter_no += 1
    if iter_no % REPORT_EVERY_ITER == 0:
        log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e", iter_no, np.mean(gen_losses), np.mean(dis_losses))
        writer.add_scalar("gen_loss", np.mean(gen_losses), iter_no)
        writer.add_scalar("dis_loss", np.mean(dis_losses), iter_no)
        gen_losses = []
        dis_losses = []
    if iter_no % SAVE_IMAGE_EVERY_ITER == 0:
        writer.add_image("fake", vutils.make_grid(gen_output_v.data[:64], normalize=True), iter_no)
        writer.add_image("real", vutils.make_grid(batch_v.data[:64], normalize=True), iter_no)

INFO: Making new env: Breakout-v0
INFO: Making new env: Assault-v0
INFO: Making new env: Pong-v0
INFO: Iter 100: gen_loss=4.343e+00, dis_loss=1.041e-01
INFO: Iter 200: gen_loss=5.992e+00, dis_loss=2.941e-02
INFO: Iter 300: gen_loss=6.711e+00, dis_loss=1.173e-02
INFO: Iter 400: gen_loss=6.968e+00, dis_loss=1.364e-01
INFO: Iter 500: gen_loss=5.740e+00, dis_loss=2.261e-02
INFO: Iter 600: gen_loss=6.081e+00, dis_loss=5.555e-03
INFO: Iter 700: gen_loss=6.442e+00, dis_loss=3.445e-03
INFO: Iter 800: gen_loss=6.410e+00, dis_loss=9.896e-03
INFO: Iter 900: gen_loss=7.377e+00, dis_loss=6.034e-02
INFO: Iter 1000: gen_loss=5.061e+00, dis_loss=1.735e-01
INFO: Iter 1100: gen_loss=5.143e+00, dis_loss=1.782e-02
INFO: Iter 1200: gen_loss=5.321e+00, dis_loss=7.372e-02
INFO: Iter 1300: gen_loss=6.175e+00, dis_loss=3.549e-02
INFO: Iter 1400: gen_loss=5.299e+00, dis_loss=1.894e-02
INFO: Iter 1500: gen_loss=5.280e+00, dis_loss=1.380e-01
INFO: Iter 1600: gen_loss=6.043e+00, dis_loss=8.529e-03
INFO: Iter 1700:

INFO: Iter 14500: gen_loss=5.471e+00, dis_loss=1.586e-01
INFO: Iter 14600: gen_loss=5.069e+00, dis_loss=1.495e-01
INFO: Iter 14700: gen_loss=5.249e+00, dis_loss=1.560e-01
INFO: Iter 14800: gen_loss=5.654e+00, dis_loss=4.492e-02
INFO: Iter 14900: gen_loss=5.084e+00, dis_loss=9.659e-02
INFO: Iter 15000: gen_loss=4.998e+00, dis_loss=1.526e-01
INFO: Iter 15100: gen_loss=6.101e+00, dis_loss=5.306e-02
INFO: Iter 15200: gen_loss=5.468e+00, dis_loss=1.335e-01
INFO: Iter 15300: gen_loss=5.743e+00, dis_loss=1.332e-01
INFO: Iter 15400: gen_loss=5.748e+00, dis_loss=5.875e-02
INFO: Iter 15500: gen_loss=5.424e+00, dis_loss=8.932e-02
INFO: Iter 15600: gen_loss=5.539e+00, dis_loss=5.332e-02
INFO: Iter 15700: gen_loss=5.642e+00, dis_loss=1.121e-02
INFO: Iter 15800: gen_loss=5.993e+00, dis_loss=2.970e-02
INFO: Iter 15900: gen_loss=5.988e+00, dis_loss=2.037e-01
INFO: Iter 16000: gen_loss=6.201e+00, dis_loss=4.464e-02
INFO: Iter 16100: gen_loss=6.468e+00, dis_loss=8.587e-02
INFO: Iter 16200: gen_loss=4.71

INFO: Iter 28900: gen_loss=6.338e+00, dis_loss=7.054e-02
INFO: Iter 29000: gen_loss=6.811e+00, dis_loss=1.046e-02
INFO: Iter 29100: gen_loss=6.445e+00, dis_loss=1.095e-01
INFO: Iter 29200: gen_loss=6.086e+00, dis_loss=6.725e-02
INFO: Iter 29300: gen_loss=5.771e+00, dis_loss=8.296e-02
INFO: Iter 29400: gen_loss=5.128e+00, dis_loss=9.809e-02
INFO: Iter 29500: gen_loss=6.587e+00, dis_loss=6.203e-02
INFO: Iter 29600: gen_loss=6.680e+00, dis_loss=4.436e-02
INFO: Iter 29700: gen_loss=5.578e+00, dis_loss=3.455e-01
INFO: Iter 29800: gen_loss=5.688e+00, dis_loss=7.083e-02
INFO: Iter 29900: gen_loss=5.334e+00, dis_loss=1.466e-01
INFO: Iter 30000: gen_loss=6.477e+00, dis_loss=1.549e-01
INFO: Iter 30100: gen_loss=6.132e+00, dis_loss=2.302e-02
INFO: Iter 30200: gen_loss=6.574e+00, dis_loss=1.508e-02
INFO: Iter 30300: gen_loss=5.989e+00, dis_loss=9.222e-02
INFO: Iter 30400: gen_loss=5.895e+00, dis_loss=1.734e-02
INFO: Iter 30500: gen_loss=6.704e+00, dis_loss=1.520e-02
INFO: Iter 30600: gen_loss=7.21

INFO: Iter 43300: gen_loss=7.152e+00, dis_loss=2.449e-02
INFO: Iter 43400: gen_loss=7.562e+00, dis_loss=1.835e-03
INFO: Iter 43500: gen_loss=7.342e+00, dis_loss=6.276e-03
INFO: Iter 43600: gen_loss=7.006e+00, dis_loss=2.333e-03
INFO: Iter 43700: gen_loss=6.716e+00, dis_loss=4.745e-02
INFO: Iter 43800: gen_loss=7.022e+00, dis_loss=7.764e-03
INFO: Iter 43900: gen_loss=7.017e+00, dis_loss=1.106e-01
INFO: Iter 44000: gen_loss=6.794e+00, dis_loss=1.025e-01
INFO: Iter 44100: gen_loss=6.672e+00, dis_loss=4.312e-02
INFO: Iter 44200: gen_loss=6.829e+00, dis_loss=1.428e-02
INFO: Iter 44300: gen_loss=7.334e+00, dis_loss=5.680e-02
INFO: Iter 44400: gen_loss=6.233e+00, dis_loss=6.560e-02
INFO: Iter 44500: gen_loss=7.449e+00, dis_loss=3.358e-02
INFO: Iter 44600: gen_loss=7.912e+00, dis_loss=6.205e-02
INFO: Iter 44700: gen_loss=6.006e+00, dis_loss=2.421e-01
INFO: Iter 44800: gen_loss=6.766e+00, dis_loss=2.004e-01
INFO: Iter 44900: gen_loss=6.867e+00, dis_loss=1.099e-01
INFO: Iter 45000: gen_loss=6.97

INFO: Iter 57700: gen_loss=8.670e+00, dis_loss=5.880e-02
INFO: Iter 57800: gen_loss=8.630e+00, dis_loss=1.105e-01
INFO: Iter 57900: gen_loss=6.480e+00, dis_loss=2.730e-02
INFO: Iter 58000: gen_loss=7.709e+00, dis_loss=1.625e-02
INFO: Iter 58100: gen_loss=7.774e+00, dis_loss=1.520e-02
INFO: Iter 58200: gen_loss=7.530e+00, dis_loss=8.327e-03
INFO: Iter 58300: gen_loss=7.534e+00, dis_loss=1.030e-02
INFO: Iter 58400: gen_loss=9.692e+00, dis_loss=1.948e-02
INFO: Iter 58500: gen_loss=1.016e+01, dis_loss=6.217e-03
INFO: Iter 58600: gen_loss=8.182e+00, dis_loss=1.818e-01
INFO: Iter 58700: gen_loss=6.620e+00, dis_loss=1.518e-01
INFO: Iter 58800: gen_loss=7.235e+00, dis_loss=3.749e-02
INFO: Iter 58900: gen_loss=7.358e+00, dis_loss=6.522e-02
INFO: Iter 59000: gen_loss=7.080e+00, dis_loss=7.320e-02
INFO: Iter 59100: gen_loss=7.716e+00, dis_loss=1.273e-02
INFO: Iter 59200: gen_loss=7.948e+00, dis_loss=3.260e-02
INFO: Iter 59300: gen_loss=8.658e+00, dis_loss=5.171e-02
INFO: Iter 59400: gen_loss=7.00

...

INFO: Iter 143500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 143600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 143700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 143800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 143900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 144900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 145000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 145100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 145

INFO: Iter 157700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 157800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 157900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 158900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 159000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 159100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 159200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 159300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 159

INFO: Iter 171900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 172900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 173

INFO: Iter 186100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186800: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 186900: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187000: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187100: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187200: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187300: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187400: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187500: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187600: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187700: gen_loss=0.000e+00, dis_loss=2.763e+01
INFO: Iter 187

KeyboardInterrupt: 